In [2]:
# Import the necessary libraries that are being used below
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, GRU
from tensorflow.keras.optimizers import Adam


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
data = pd.read_csv("Multi_labeled_CAN_dataset.csv")

label_encoder = LabelEncoder()
data['ID'] = label_encoder.fit_transform(data['ID'])
data['DATA'] = label_encoder.fit_transform(data['DATA'])
X = data[['Timestamp', 'ID', 'DLC', 'DATA']].values
y = data['Class'].values

# Normalize the features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Reshape the input data to have a temporal dimension
X = X.reshape(X.shape[0], 1, X.shape[1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [6]:
X_train.shape

(4152095, 1, 4)

In [7]:
from tensorflow.keras.models import load_model

# Define the path to the saved Keras model file
model_file_path = 'Multiclass_model.h5'

# Load the saved Keras model
model = load_model(model_file_path)

# Now you can use the `new_model` variable to make predictions or perform other operations
# For example:
# y_pred = new_model.predict(X_test)
# print(y_pred)


In [8]:
# Import necessary libraries for memory profiling and timing
import os
# import timeit

# Predict accuracy on test dataset
accuracy = model.evaluate(X_test, y_test)
test_accuracy_percentage = accuracy[1] * 100  # Convert accuracy to percentage
print("Test Accuracy:", "{:.2f}%".format(test_accuracy_percentage))  # Display accuracy as percentage


14417/14417 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.8812 - loss: 0.3399
Test Accuracy: 88.04%


In [10]:
# Measure memory usage of the model
non_quantized_model_size = os.path.getsize('Multiclass_model.h5')
print("non_quantized model is %d bytes" % non_quantized_model_size)


non_quantized model is 200440 bytes


In [109]:
from timeit import default_timer as timer
# Select a single data point from the test dataset for prediction
single_test_data = X_test[0].reshape(1, 1, X.shape[2])  # Reshape for single data point

# Measure prediction time
start_time = timer()
p = model.predict(single_test_data)
end_time = timer()
prediction_time = (end_time - start_time) * 1000

print("Prediction Time for a Single Data Point:", "{:.2f} ms".format(prediction_time))  # Display prediction time in milliseconds


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction Time for a Single Data Point: 102.95 ms


In [ ]:
model.dtype

'float32'

## **TFLite Default**

In [21]:
# Assuming 'model' is your original TensorFlow/Keras model
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS      # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the TFLite model to a file
with open("c_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\routs\AppData\Local\Temp\tmp3c0cpix9\assets


INFO:tensorflow:Assets written to: C:\Users\routs\AppData\Local\Temp\tmp3c0cpix9\assets


Saved artifact at 'C:\Users\routs\AppData\Local\Temp\tmp3c0cpix9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1, 4), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  1699618332624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618331664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618333008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618333776: TensorSpec(shape=(), dtype=tf.resource, name=None)


## **tflite default delay**

In [32]:
# Load the TFLite model
c_model = tf.lite.Interpreter('c_model.tflite')
c_model.allocate_tensors()

test_image = np.expand_dims(X_test[0], axis=0).astype(np.float32)

# Get indexes of the input and output tensors
c_model_input_index = c_model.get_input_details()[0]["index"]
c_model_output_index = c_model.get_output_details()[0]["index"]

start1 = timer()
c_model.set_tensor(c_model_input_index, test_image)
c_model.invoke()
predictions_default = c_model.get_tensor(c_model_output_index)
end1 = timer()

prediction_default_time = (end1-start1) * 1000  ## in ms
print("TFLite Model Prediction Time for a Single Data Point:", "{:.2f} ms".format(prediction_default_time))

TFLite Model Prediction Time for a Single Data Point: 1.07 ms


## **TFlite default accuracy**


In [110]:
# # Evaluate testing accuracy of the TFLite model
correct_count = 0
total_count = len(X_test)

# def f1(p):
#   return 1 if p >= 0.5 else 0

for i, x_value in enumerate(X_test):
    x_value_tensor = tf.convert_to_tensor(x_value, dtype=np.float32)
    x_value_tensor = np.expand_dims(x_value_tensor, axis=0)
    c_model.set_tensor(c_model_input_index, x_value_tensor)
    c_model.invoke()
    c_model_prediction = c_model.get_tensor(c_model_output_index)[0]

    # Compare prediction with ground truth label
    if (np.argmax(c_model_prediction) == y_test[i]):
        correct_count += 1

accuracy_percentage = (correct_count / total_count) * 100

# Calculate and print testing accuracy of the TFLite model

print("TFLite Model Testing Accuracy:", "{:.2f}%".format(accuracy_percentage))

TFLite Model Testing Accuracy: 88.04%


In [35]:
# Measure memory usage of the TFLite model
c_model_default_size = os.path.getsize("c_model.tflite")
print("floating point 32 quantized model is %d bytes" % c_model_default_size)

floating point 32 quantized model is 68268 bytes


## **TFLite with float16**

In [36]:
# Create a TFLite converter from the Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply optimizations and specify supported ops
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS      # Enable TensorFlow ops
]
converter.target_spec.supported_types = [tf.float16]

# Disable experimental tensor list lowering
converter._experimental_lower_tensor_list_ops = False

# Convert the model to TensorFlow Lite format
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model_fp16.tflite', 'wb') as f:
    f.write(tflite_model)

# print("TFLite model conversion completed successfully.")

INFO:tensorflow:Assets written to: C:\Users\routs\AppData\Local\Temp\tmp8w61spof\assets


INFO:tensorflow:Assets written to: C:\Users\routs\AppData\Local\Temp\tmp8w61spof\assets


Saved artifact at 'C:\Users\routs\AppData\Local\Temp\tmp8w61spof'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1, 4), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  1699618332624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618331664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618333008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618333776: TensorSpec(shape=(), dtype=tf.resource, name=None)


## **fp16 delay**

In [61]:
# Load the TFLite model
fp16_model = tf.lite.Interpreter('model_fp16.tflite')
fp16_model.allocate_tensors()

# Get indexes of the input and output tensors
fp16_model_input_index = fp16_model.get_input_details()[0]["index"]
fp16_model_output_index = fp16_model.get_output_details()[0]["index"]

start2 = timer()
fp16_model.set_tensor(fp16_model_input_index, test_image)
fp16_model.invoke()
predictions_fp16 = fp16_model.get_tensor(fp16_model_output_index)
end2 = timer()
prediction_fp16_time = (end2-start2) * 1000  ## in ms
# predictions

print("TFLite Model Prediction Time for a Single Data Point:", "{:.2f} ms".format(prediction_fp16_time))

TFLite Model Prediction Time for a Single Data Point: 0.99 ms


## **fp16 accuracy**

In [63]:
# Evaluate testing accuracy of the TFLite model
correct_count = 0

for i, x_value in enumerate(X_test):
    x_value_tensor = tf.convert_to_tensor(x_value, dtype=np.float32)
    x_value_tensor = np.expand_dims(x_value_tensor, axis=0)
    fp16_model.set_tensor(fp16_model_input_index, x_value_tensor)
    fp16_model.invoke()
    fp16_model_prediction = fp16_model.get_tensor(fp16_model_output_index)[0]

   
    # Compare prediction with ground truth label
    if (np.argmax(fp16_model_prediction) == y_test[i]):
        correct_count += 1

accuracy_percentage = (correct_count / total_count) * 100

# Calculate and print testing accuracy of the TFLite model

print("TFLite fp16 Model Testing Accuracy:", "{:.2f}%".format(accuracy_percentage))

TFLite fp16 Model Testing Accuracy: 88.01%


## **fp 16 tflite memory**

In [64]:
# Measure memory usage of the TFLite model
fp16_quantized_model_size = os.path.getsize("model_fp16.tflite")
print("floating point 16 quantized model is %d bytes" % fp16_quantized_model_size)

floating point 16 quantized model is 41396 bytes


# **TFLite with int8**

In [65]:
def representative_dataset():
  for value in X_test:
     yield [np.array(value, dtype=np.float32, ndmin=3)]

# Create a TFLite converter from the Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply optimizations and specify supported ops
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS, # enable TensorFlow ops.
  tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Disable experimental tensor list lowering
converter._experimental_lower_tensor_list_ops = False
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Convert the model to TensorFlow Lite format
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model_int8.tflite', 'wb') as f:
    f.write(tflite_model)

# print("TFLite model conversion completed successfully.")

INFO:tensorflow:Assets written to: C:\Users\routs\AppData\Local\Temp\tmpt8xgnhlp\assets


INFO:tensorflow:Assets written to: C:\Users\routs\AppData\Local\Temp\tmpt8xgnhlp\assets


Saved artifact at 'C:\Users\routs\AppData\Local\Temp\tmpt8xgnhlp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1, 4), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  1699618332624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618331664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618333008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618330320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1699618333776: TensorSpec(shape=(), dtype=tf.resource, name=None)


c:\Users\routs\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\lite\python\convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


## **int8 tflite delay**

In [84]:
# Load the TFLite model
int8_model = tf.lite.Interpreter('model_int8.tflite')
int8_model.allocate_tensors()

test_image = np.expand_dims(X_test[0], axis=0).astype(np.float32)

# Get indexes of the input and output tensors
int8_model_input_index = int8_model.get_input_details()[0]["index"]
int8_model_output_index = int8_model.get_output_details()[0]["index"]

start3 = timer()
int8_model.set_tensor(int8_model_input_index, test_image)
int8_model.invoke()
predictions = int8_model.get_tensor(int8_model_output_index)
end3 = timer()
prediction_int8_time = (end3 -start3) * 1000  ## in ms
# predictions

print("TFLite Model Prediction Time for a Single Data Point:", "{:.2f} ms".format(prediction_int8_time))

TFLite Model Prediction Time for a Single Data Point: 0.89 ms


## **int8 tflite aaccuracy**

In [86]:

# # Evaluate testing accuracy of the TFLite model
correct_count = 0

for i, x_value in enumerate(X_test):
    x_value_tensor = tf.convert_to_tensor(x_value, dtype=np.float32)
    x_value_tensor = np.expand_dims(x_value_tensor, axis=0)
    int8_model.set_tensor(int8_model_input_index, x_value_tensor)
    int8_model.invoke()
    int8_model_prediction = int8_model.get_tensor(int8_model_output_index)[0]

    
    # Compare prediction with ground truth label
    if (np.argmax(int8_model_prediction) == y_test[i]):
        correct_count += 1

accuracy_percentage = (correct_count / total_count) * 100

# Calculate and print testing accuracy of the TFLite model

print("TFLite int8 Model Testing Accuracy:", "{:.2f}%".format(accuracy_percentage))

TFLite int8 Model Testing Accuracy: 83.63%


## **int8 tflite memory**

In [87]:
# import os
# Measure memory usage of the TFLite model
int8_quantized_model_size = os.path.getsize("model_int8.tflite")
print("integer point 8 quantized model is %d bytes" % int8_quantized_model_size)

integer point 8 quantized model is 40328 bytes
